In [4]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def crawl_with_selenium(target_count=300):
    """Selenium을 사용한 네이버 뉴스 크롤링"""
    
    print(f"Selenium을 사용해서 {target_count}개 기사를 수집합니다...")
    
    # 웹드라이버 시작
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)
    
    try:
        # 네이버 뉴스 페이지로 이동
        url = 'https://news.naver.com/breakingnews/section/103/241'
        driver.get(url)
        time.sleep(3)
        
        # 더보기 버튼을 클릭해서 기사를 더 로드
        while True:
            # 현재 페이지의 기사 개수 확인
            articles = driver.find_elements(By.CSS_SELECTOR, 'a.sa_text_title')
            current_count = len(articles)
            
            print(f"현재 로드된 기사 수: {current_count}개")
            
            # 목표 개수에 도달하면 중단
            if current_count >= target_count:
                print(f"목표 개수 {target_count}개에 도달했습니다!")
                break
            
            try:
                # 더보기 버튼 찾기 (실제 클래스명 사용)
                more_button = driver.find_element(By.CSS_SELECTOR, '.section_more_inner_CONTENT_LIST_LOAD_MORE_BUTTON')
                
                # 버튼이 보이도록 스크롤
                driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
                time.sleep(1)
                
                # 더보기 버튼 클릭
                more_button.click()
                print("더보기 버튼을 클릭했습니다.")
                
                # 새로운 기사 로딩 대기
                time.sleep(3)
                
            except Exception as e:
                # 다른 가능한 더보기 버튼 셀렉터들 시도
                alternative_selectors = [
                    'button[class*="more"]',
                    'button[class*="MORE"]', 
                    'a[class*="more"]',
                    '.section_more_btn',
                    '[data-persist-able="false"]'
                ]
                
                button_found = False
                for selector in alternative_selectors:
                    try:
                        more_button = driver.find_element(By.CSS_SELECTOR, selector)
                        driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
                        time.sleep(1)
                        more_button.click()
                        print(f"더보기 버튼을 클릭했습니다. (셀렉터: {selector})")
                        button_found = True
                        break
                    except Exception:
                        continue
                
                if not button_found:
                    print("더보기 버튼을 찾을 수 없습니다.")
                    print("더 이상 로드할 기사가 없는 것 같습니다.")
                    break
                
                # 새로운 기사 로딩 대기
                time.sleep(3)
                
            except Exception as e:
                print(f"더보기 버튼을 찾을 수 없습니다: {e}")
                print("더 이상 로드할 기사가 없는 것 같습니다.")
                break
        
        # 최종 기사 리스트 수집
        final_articles = driver.find_elements(By.CSS_SELECTOR, 'a.sa_text_title')
        
        # 목표 개수만큼만 선택
        selected_articles = final_articles[:target_count]
        
        print(f"총 {len(selected_articles)}개 기사 URL을 수집했습니다.")
        
        # URL 리스트 생성
        href_list = []
        for article in selected_articles:
            href = article.get_attribute('href')
            if href:
                href_list.append(href)
        
        print(f"유효한 URL {len(href_list)}개를 수집했습니다.")
        
        return href_list
        
    except Exception as e:
        print(f"Selenium 크롤링 중 오류: {e}")
        return []
    
    finally:
        driver.quit()

def crawl_news_content(href_list):
    """기사 내용 크롤링"""
    
    print("기사 내용을 크롤링합니다...")
    data_list = []
    
    for i, href in enumerate(href_list):
        try:
            print(f"진행률: {i+1}/{len(href_list)} - 기사 크롤링 중...")
            
            # URL 요청
            res = req.get(href)
            
            # HTML 파싱
            html = bs(res.text, 'lxml')
            
            # 제목 추출
            try:
                title_element = html.select_one('#title_area > span')
                title = title_element.text.strip() if title_element else "제목 없음"
            except:
                title = "제목 추출 실패"
            
            # 내용 추출
            try:
                content_element = html.select_one('#dic_area')
                content = content_element.text.strip() if content_element else "내용 없음"
            except:
                content = "내용 추출 실패"
            
            # 데이터 저장
            data_list.append([title, content, href])
            
            # 요청 간격 조절 (서버 부하 방지)
            time.sleep(0.5)
            
        except Exception as e:
            print(f"기사 {i+1} 크롤링 중 오류: {e}")
            # 오류가 발생해도 계속 진행
            data_list.append(["오류 발생", "오류 발생", href])
            continue
    
    return data_list

# ========== 메인 실행 코드 ==========

if __name__ == "__main__":
    # 1단계: Selenium으로 기사 URL 수집 (300개)
    print("=== 1단계: 기사 URL 수집 ===")
    href_list = crawl_with_selenium(target_count=300)
    
    if not href_list:
        print("URL 수집에 실패했습니다.")
        exit()
    
    # 2단계: 각 기사 내용 크롤링
    print("\n=== 2단계: 기사 내용 크롤링 ===")
    data_list = crawl_news_content(href_list)
    
    # 3단계: 데이터프레임 생성 및 저장
    print("\n=== 3단계: 데이터 저장 ===")
    news_df = pd.DataFrame(data_list, columns=['뉴스제목', '내용', '페이지주소'])
    
    # 결과 출력
    print(f"총 {len(news_df)}개 기사를 수집했습니다.")
    print("\n=== 수집 결과 미리보기 ===")
    print(news_df.head())
    
    # CSV 파일로 저장
    news_df.to_csv('naver_news_300.csv', index=False, encoding='utf-8-sig')
    print(f"\n데이터가 'naver_news_300.csv' 파일로 저장되었습니다.")
    
    # 기본 통계 출력
    print(f"\n=== 크롤링 통계 ===")
    print(f"전체 기사 수: {len(news_df)}")
    print(f"평균 제목 길이: {news_df['뉴스제목'].str.len().mean():.1f}자")
    print(f"평균 내용 길이: {news_df['내용'].str.len().mean():.1f}자")
    
    print("\n크롤링이 완료되었습니다! 🎉")

=== 1단계: 기사 URL 수집 ===
Selenium을 사용해서 300개 기사를 수집합니다...
현재 로드된 기사 수: 36개
더보기 버튼을 클릭했습니다. (셀렉터: a[class*="more"])
현재 로드된 기사 수: 72개
더보기 버튼을 클릭했습니다. (셀렉터: a[class*="more"])
현재 로드된 기사 수: 108개
더보기 버튼을 클릭했습니다. (셀렉터: a[class*="more"])
현재 로드된 기사 수: 144개
더보기 버튼을 클릭했습니다. (셀렉터: a[class*="more"])
현재 로드된 기사 수: 180개
더보기 버튼을 클릭했습니다. (셀렉터: a[class*="more"])
현재 로드된 기사 수: 216개
더보기 버튼을 클릭했습니다. (셀렉터: a[class*="more"])
현재 로드된 기사 수: 252개
더보기 버튼을 클릭했습니다. (셀렉터: a[class*="more"])
현재 로드된 기사 수: 288개
더보기 버튼을 클릭했습니다. (셀렉터: a[class*="more"])
현재 로드된 기사 수: 324개
목표 개수 300개에 도달했습니다!
총 300개 기사 URL을 수집했습니다.
유효한 URL 300개를 수집했습니다.

=== 2단계: 기사 내용 크롤링 ===
기사 내용을 크롤링합니다...
진행률: 1/300 - 기사 크롤링 중...
진행률: 2/300 - 기사 크롤링 중...
진행률: 3/300 - 기사 크롤링 중...
진행률: 4/300 - 기사 크롤링 중...
진행률: 5/300 - 기사 크롤링 중...
진행률: 6/300 - 기사 크롤링 중...
진행률: 7/300 - 기사 크롤링 중...
진행률: 8/300 - 기사 크롤링 중...
진행률: 9/300 - 기사 크롤링 중...
진행률: 10/300 - 기사 크롤링 중...
진행률: 11/300 - 기사 크롤링 중...
진행률: 12/300 - 기사 크롤링 중...
진행률: 13/300 - 기사 크롤링 중...
진행률: 14/300 - 기사 크롤링 중...
진행률

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('naver_news_300.csv')
df

,뉴스제목,내용,페이지주소
0,“이제 위내시경 매년 안 받아도 되겠네?”…새로 바뀌는 ‘위암 검진 권고안’ 보니,기사 내용과 무관한 이미지.클립아트코리아[서울경제] 국립암센터가 1일 공청회를 열고...,https://n.news.naver.com/mnews/article/011/000...
1,위내시경 검진 주기 2년으로 바뀌나…'국가 위암 검진 권고' 개정,국립암센터 지난 1일 '국가위암검진 권고안 개정 공청회' 개최검진 권고 연령 40~...,https://n.news.naver.com/mnews/article/421/000...
2,"""건강에 좋다"" SNS서 뜬 코르티솔칵테일…과연 그럴까","""과학적 근거 없어…'부신 피로'도 근거 없는 주장""\n\n\n\n코코넛 워터에 오...",https://n.news.naver.com/mnews/article/296/000...
3,65세 미만 ‘젊은층’ 심장마비 원인…男女가 너~무 다르다?,심장마비 원인 중 죽상동맥경화증에 의한 혈전(피떡)…男은 75%이나 女는 47%에 ...,https://n.news.naver.com/mnews/article/296/000...
4,"[열린송현] 과학기술은 국경을 넘고, 협력은 경쟁력이 된다","■김일두 KAIST 신소재공학과 교수AI등 글로벌 공급망과 밀접 연결과기부, 타부처...",https://n.news.naver.com/mnews/article/011/000...
...,...,...,...
295,“하루 동안 벼락이 733번?” 이틀 후 ‘이것’ 앓는 환자 속출…무슨 일?,비구름 속 꽃가루·곰팡이가 잘게 쪼개져 호흡기 질환 유발\n\n\n\n9일 중국 베...,https://n.news.naver.com/mnews/article/296/000...
296,"휴젤, '젊은 CEO' 장두현 대표 선임…미래 가치 제고",장두현 휴젤 대표. (휴젤 제공)(서울=뉴스1) 문대현 기자 = 휴젤(145020)...,https://n.news.naver.com/mnews/article/421/000...
297,"GC녹십자웰빙, '통증대가 심포지엄'서 최신 치료 트렌드 공유","(왼쪽부터) 김완호 원장(김완호정형외과의원), 채상한 원장(송도마디재활의학과의원),...",https://n.news.naver.com/mnews/article/421/000...
298,"유한양행, 휴이노와 AI 모니터링 솔루션 '메모큐' 국내 공급 계약",휴이노가 개발 유한양행이 판매제품 단위에서 솔루션 단위 확대\n\n\n\n휴이노 길...,https://n.news.naver.com/mnews/article/119/000...


In [7]:
from konlpy.tag import Okt
okt=Okt()

In [8]:
stopwords_df = pd.read_csv(r"ko-stopwords.csv")
stopwords_df

,stopwords
0,가
1,가까스로
2,가령
3,각
4,각각
...,...
600,진짜
601,두
602,게
603,요


In [9]:
stopwords = set(stopwords_df['stopwords']) #집합 변경
stopwords

{'가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '게',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '단지',
 '답다',
 '당신',
 '당장',
 '대로 하다',
 '대하면',
 '대하여',
 '대해 말하자면',
 '대해서',
 

In [10]:
'Noun', 'Adjective', 'Verb'

('Noun', 'Adjective', 'Verb')

In [11]:
def okt_pos_tagging(string) :
    pos_words = okt.pos(string, stem =True, norm=True)
    result = [word for word, tag in pos_words if word not in stopwords if tag in {'Noun', 'Verb'} ]
    return result

In [12]:
okt_pos_tagging(df['내용'][0])

['기사',
 '내용',
 '무관',
 '이미지',
 '클립아트',
 '코리아',
 '서울',
 '경제',
 '국립',
 '센터',
 '공청회',
 '열다',
 '국제',
 '표준',
 '방법론',
 '적용',
 '마련',
 '국가',
 '위암',
 '검진',
 '권고',
 '안',
 '공개',
 '검토',
 '밝히다',
 '국가',
 '위암',
 '검진',
 '한국',
 '위암',
 '생존',
 '율',
 '세계',
 '최고',
 '수준',
 '끌다',
 '올리다',
 '크다',
 '역할',
 '해오다',
 '국립',
 '센터',
 '관련',
 '학회',
 '첫',
 '권고',
 '안',
 '개발',
 '이후',
 '차례',
 '개정',
 '개정',
 '이뤄지다',
 '개정',
 '권고',
 '안다',
 '세계보건기구',
 '코크',
 '란',
 '주요',
 '국제',
 '구가',
 '채택',
 '권고',
 '평가',
 '개발',
 '등급',
 '기준',
 '방법론',
 '기반',
 '개발',
 '돼다',
 '늘다',
 '근거',
 '확실성',
 '질',
 '이익',
 '위해',
 '종합',
 '고려',
 '권고',
 '안',
 '만들다',
 '체계',
 '접근',
 '법',
 '가이드라인',
 '개발',
 '국제',
 '표준',
 '통한',
 '개정안',
 '핵심',
 '내용',
 '위',
 '내시경',
 '검진',
 '권고',
 '연령',
 '세',
 '세',
 '위',
 '내시경',
 '검진',
 '주기',
 '위장',
 '촬영',
 '검사',
 '경우',
 '시행',
 '기존',
 '의사',
 '상담',
 '후',
 '조건',
 '부',
 '권고',
 '포함',
 '돼다',
 '위장',
 '촬영',
 '검사',
 '위',
 '내시경',
 '상황',
 '한층',
 '제한',
 '날',
 '공청회',
 '위',
 '내시경',
 '검진',
 '질',
 '관리',
 '고령',
 '층',
 '검진',
 '방안',
 '위험',
 '검진',
 '전략',
 '주요'

In [13]:
import pandas as pd
from tqdm.auto import tqdm

In [14]:
tqdm.pandas()